In [7]:
from tqdm import tqdm
import weaviate
import os
import requests
client = weaviate.connect_to_custom(
    http_host=os.getenv("WEAVIATE_HTTP_HOST"),
    http_port=os.getenv("WEAVIATE_HTTP_PORT"),
    http_secure=False,
    grpc_host=os.getenv("WEAVIATE_GRPC_HOST"),
    grpc_port=os.getenv("WEAVIATE_GRPC_PORT"),
    grpc_secure=False,
)

In [9]:
from weaviate.classes.config import Configure, Property, DataType
client.is_ready()

True

In [32]:
embed_url = os.getenv("EMBEDDING_URL")
rerank_url = os.getenv("RERANK_URL")
doc_url = './intent_RAG-update.txt'
collection_name = "intent_RAG_1"

In [29]:
with open(doc_url, 'r') as file:
    lines = [line.replace('\n#','#').replace(":","").split('\n') for line in file.read().split('\n\n')]
lines

[['# BOOKING SERVICE INTENT - Đặt lịch dọn nhà',
  'booking_service',
  '- đặt lịch dọn nhà',
  '- Đặt lịch dọn nhà cho tôi',
  '- lịch dọn nhà',
  '- muốn đặt lịch dọn nhà',
  '- cần đặt lịch dọn nhà',
  '- tôi muốn đặt lịch dọn nhà',
  '- anh muốn đặt lịch dọn nhà',
  '- em muốn đặt lịch dọn nhà',
  '- đặt lịch dọn nhà giúp tôi',
  '- đặt lịch dọn nhà hộ tôi',
  '- book lịch dọn nhà',
  '- đặt lịch dọn',
  '- lịch dọn'],
 ['# BOOKING SERVICE INTENT - Đặt lịch dọn dẹp',
  'booking_service',
  '- Tôi muốn đặt lịch dọn dẹp nhà',
  '- đặt lịch dọn dẹp',
  '- lịch dọn dẹp',
  '- muốn đặt lịch dọn dẹp',
  '- cần đặt lịch dọn dẹp',
  '- tôi muốn đặt lịch dọn dẹp'],
 ['# BOOKING SERVICE INTENT - Book dịch vụ',
  'booking_service',
  '- Tôi muốn book dịch vụ cleaning',
  '- book dịch vụ dọn nhà',
  '- Tôi muốn book cleaning service',
  '- Book dịch vụ dọn dẹp nhà',
  '- Tôi cần đặt lịch dọn dẹp',
  '- Đặt dịch vụ vệ sinh nhà cửa',
  '- Tôi muốn book cleaning service',
  '- Cần đặt lịch dọn dẹ

In [5]:
def custom_vectorizer(data: dict) -> list[float]:
    embed_payload = {
        "input": data
    }
    response = requests.post(f"{embed_url}", json=embed_payload)
    vector_embeded = response.json()["data"][0]["embedding"]
    return vector_embeded

In [8]:
collection = client.collections.get("a")
for intent in lines:
    for i in tqdm(range(len(intent))):
        main_intent=intent[1]
        description_intent = intent[i]
        collection.data.insert(  
            properties={
                "vectorizer": "none",
                "main_intent": main_intent,
                "description_intent": description_intent
            },
            vector=custom_vectorizer(description_intent)
        )

100%|██████████| 42/42 [00:02<00:00, 18.25it/s]


In [112]:
from weaviate.classes.query import MetadataQuery
query = "không đặt nữa tư vấn cho tôi"
response = client.collections.get("Article1").query.hybrid(
    query=query,
    alpha=0.7,
    vector=custom_vectorizer(query),
    limit=20,
    include_vector=False,
    return_metadata=MetadataQuery(
        score=True, 
        explain_score=True,
        distance=True,
        last_update_time=True
        ),
)

In [113]:
for o in response.objects:
    print(o.properties)
    print(o.metadata.score)           # Higher = more confident

{'main_intent': 'booking_service', 'description_intent': 'Không em', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general questions)'}
0.7657371759414673
{'main_intent': 'booking_service', 'description_intent': 'other_intent:', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general questions)'}
0.5559930205345154
{'main_intent': 'booking_service', 'description_intent': 'Tính giá dọn dẹp nhà cho tôi', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general questions)'}
0.5195099115371704
{'main_intent': 'booking_service', 'description_intent': 'Không cần đâu em', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general questions)'}
0.5152219533920288
{'main_intent': 'booking_service', 'description_intent': 'Dịch vụ có những gì', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general question

In [93]:
requests.post(f"{rerank_url}", json={
    "query": query,
    "documents":[o.properties.get("main_intent")+" "+o.properties.get("description_intent") for o in response.objects],
    "top_n": 10,
    "model": "Qwen/Qwen3-Embedding-0.6B",
  }).json().get("results")

[{'index': 3,
  'document': {'text': 'booking_service Không cần đâu em'},
  'relevance_score': 0.6516455411911011},
 {'index': 14,
  'document': {'text': 'booking_service Khỏi em\nother_intent- À khỏi \nother_intent- Không\nother_intent- không cần thêm\nother_intent- No\nother_intent- Not\nother_intent- No thanks'},
  'relevance_score': 0.6352251172065735},
 {'index': 0,
  'document': {'text': 'booking_service Không em'},
  'relevance_score': 0.6336103081703186},
 {'index': 11,
  'document': {'text': 'booking_service Hi bạn'},
  'relevance_score': 0.5906387567520142},
 {'index': 4,
  'document': {'text': 'booking_service Dịch vụ có những gì'},
  'relevance_score': 0.5883540511131287},
 {'index': 18,
  'document': {'text': 'booking_service Xin chào'},
  'relevance_score': 0.5807054042816162},
 {'index': 8,
  'document': {'text': 'booking_service Chào em'},
  'relevance_score': 0.5717272162437439},
 {'index': 12,
  'document': {'text': 'booking_service Có bảo hiểm không'},
  'relevance_s

In [100]:
AA = client.collections.get("Article1").query.near_text(
    # near_vector=custom_vectorizer(query),
    query=query,
    limit=10,
    include_vector=False,
)

WeaviateQueryError: Query call with protocol GRPC search failed with message explorer: get class: vectorize params: could not vectorize input for collection Article1 with search-type nearText. Make sure a vectorizer module is configured for this collection.

In [97]:
AA.objects

[GenerativeObject(uuid=_WeaviateUUIDInt('3f7eb9a3-d264-4bf3-ac20-09a21aff3116'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'main_intent': 'booking_service', 'description_intent': 'Không em', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general questions)'}, references=None, vector={}, collection='Article1'),
 GenerativeObject(uuid=_WeaviateUUIDInt('bd661d47-1d97-481a-8bac-524885d86f8f'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'main_intent': 'booking_service', 'description_intent': 'other_intent:', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general questions)'}, references=None, vector={}, collection='Article1'),
 Generative

In [81]:
import requests
from tqdm import tqdm
import re
from weaviate import Client # Ví dụ với thư viện Weaviate, bạn có thể thay thế bằng thư viện của Vector DB bạn dùng.

# Giả định:
# client = Client(...)
# collection = client.collections.get("a")

doc_url = './intent_RAG-update.txt' # Tên file dữ liệu của bạn
def custom_vectorizer(text: str) -> list[float]:
    """Hàm gọi API để lấy embedding cho một đoạn văn bản"""
    embed_payload = {
        "input": text
    }
    try:
        response = requests.post(f"{embed_url}", json=embed_payload)
        response.raise_for_status() # Kiểm tra lỗi HTTP
        return response.json()["data"][0]["embedding"]
    except Exception as e:
        print(f"Lỗi khi vector hóa: {e}")
        return None

# Mở file và xử lý dữ liệu một cách cẩn thận
with open(doc_url, 'r', encoding='utf-8') as file:
    content = file.read()

# Phân tách file thành các khối dựa trên tiêu đề ý định
intent_blocks = re.split(r'#\s*([A-Z_]+\s*INTENT.*)', content)[1:]

data_to_ingest = []
for i in range(0, len(intent_blocks), 2):
    header = intent_blocks[i].strip()
    block_content = intent_blocks[i+1].strip()

    # Tách lấy main_intent và sub_intent từ header
    match = re.search(r'INTENT\s*-\s*(.*)', header)
    if match:
        main_intent = "booking_service" # Bạn có thể trích xuất thông minh hơn nếu cần
        sub_intent = match.group(1).strip()
    else:
        continue

    # Tách nội dung thành từng câu riêng lẻ
    phrases = [line.strip() for line in block_content.split('\n- ') if line.strip() and not line.strip().startswith('booking_service:')]

    for phrase in phrases:
        data_to_ingest.append({
            "text": phrase,
            "main_intent": main_intent,
            "sub_intent": sub_intent
        })

# Bây giờ, vector hóa và chèn từng câu một vào database
for item in tqdm(data_to_ingest, desc="Ingesting data"):
    vector = custom_vectorizer(item["text"])
    if vector:
        collection = client.collections.get("Article1")
        collection.data.insert(
            properties={
                "main_intent": item["main_intent"],
                "sub_intent": item["sub_intent"],
                "description_intent": item["text"]
            },
            vector=vector
        )

print("Hoàn tất quá trình nhập dữ liệu.")

Ingesting data: 100%|██████████| 34/34 [00:01<00:00, 19.36it/s]

Hoàn tất quá trình nhập dữ liệu.


In [35]:
collection.name

'A'

In [ ]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances

client.collections.create(
    "Article1",
    vector_config=[
        Configure.Vectors.self_provided(
            name="custom_vector",
            vector_index_config=Configure.VectorIndex.hnsw(
                distance_metric=VectorDistances.COSINE
            ),
            
        ),
    ],
    properties=[ 
        Property(name="title", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
    ],
)

In [101]:
response.objects

[Object(uuid=_WeaviateUUIDInt('3f7eb9a3-d264-4bf3-ac20-09a21aff3116'), metadata=MetadataReturn(creation_time=None, last_update_time=datetime.datetime(2025, 8, 22, 3, 46, 15, 772000, tzinfo=datetime.timezone.utc), distance=None, certainty=None, score=0.7657371759414673, explain_score='\nHybrid (Result Set keyword,bm25) Document 3f7eb9a3-d264-4bf3-ac20-09a21aff3116: original score 0.8293658, normalized score: 0.0657372 - \nHybrid (Result Set vector,hybridVector) Document 3f7eb9a3-d264-4bf3-ac20-09a21aff3116: original score -20.048994, normalized score: 0.7', is_consistent=None, rerank_score=None), properties={'main_intent': 'booking_service', 'description_intent': 'Không em', 'title': None, 'body': None, 'sub_intent': 'Tất cả các intent khác (pricing, greeting, general questions)'}, references=None, vector={}, collection='Article1'),
 Object(uuid=_WeaviateUUIDInt('bd661d47-1d97-481a-8bac-524885d86f8f'), metadata=MetadataReturn(creation_time=None, last_update_time=datetime.datetime(2025, 

In [103]:
client.collections.exists("Article1")

True

In [109]:
from weaviate.classes.query import Filter

collection = client.collections.get("Article1")
result = collection.data.delete_many(
    where=Filter.by_property("main_intent").like("booking_service"),
    dry_run=True,
    verbose=True
)

print(result)

DeleteManyReturn(failed=0, matches=34, objects=[DeleteManyObject(uuid=_WeaviateUUIDInt('bd661d47-1d97-481a-8bac-524885d86f8f'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('b6a165a4-16c3-4746-80e0-a38d1d47f44f'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('d2107173-c723-4d0a-9958-ba79e25e45e5'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('d5e45dec-d2c6-4086-b37a-8481e5e05c70'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('960a8cc5-5568-4247-a3b5-3ea80be371b4'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('4acc165d-09a3-4daa-bf21-d62ce2fed39e'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('1d66cc6d-4161-486a-8ed3-4431b988120f'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('3670a35f-3ede-4e20-a31d-53be9b60ce73'), successful=True, error=None), DeleteManyObject(uuid=_WeaviateUUIDInt('af015e53-8e88-4a3e-96ba-1f89699a

In [9]:
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append('../')
from src.weaviate.weaviate_client import WeaviateClient
weaviate_client = WeaviateClient()

/var/folders/v9/5vpscx8s60134_lcsn5c_v1w0000gq/T/ipykernel_26626/2503439863.py:6: ResourceWarning: unclosed <socket.socket fd=81, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.88.251', 58152), raddr=('192.168.88.165', 3340)>
  weaviate_client = WeaviateClient()
